In [0]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import csv
from collections import OrderedDict

BATCH_SIZE = 20
NUM_EPOCHS = 20

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
#drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
os.chdir("/content/gdrive/My Drive/MyNet")

定义数据读取格式

In [0]:
class ClfDataset(Dataset):

    def __init__(self,train_INFO,train_DATA,crop_size=32, transform=None):

        index = []
        index += list(train_INFO.index)
        self.index = tuple(sorted(index)) 
        self.label = train_INFO['lable']
        self.transform = transform
        self.crop_size = crop_size

    def __getitem__(self,item):
        name = train_INFO.loc[self.index[item], 'name']
        with np.load(os.path.join(train_DATA, '%s.npz' % name)) as npz:
            seg=npz['seg']+0
            #seg=seg.float()
            voxel=npz['voxel']
            #voxel=voxel.float()
        #取crop_size*crop_size*crop_size
        seg= seg[50 - self.crop_size // 2:50 + self.crop_size // 2,
                50 - self.crop_size // 2:50 + self.crop_size // 2,
                50 - self.crop_size // 2:50 + self.crop_size // 2]
        voxel=voxel[50 - self.crop_size // 2:50 + self.crop_size // 2,
                50 - self.crop_size // 2:50 + self.crop_size // 2,
                50 - self.crop_size // 2:50 + self.crop_size // 2]
        data=np.multiply(seg,voxel)
        #翻转
        # z=np.random.randint(0,2)
        # x=np.random.randint(0,2)
        # y=np.random.randint(0,2)
        # if(z==1):
        #   data=np.flip(data,0)
        # if(x==1):
        #   data=np.flip(data,1)
        # if(y==1):
        #   data=np.flip(data,2)
        # data=data.copy()
        #90度旋转
        # angle = np.random.randint(4, size=3)
        # X=np.rot90(data,angle[0],axes=(0,1))
        # Y=np.rot90(X,angle[1],axes=(0,2))
        # Z=np.rot90(Y,angle[2],axes=(1,2))
        # data=Z.copy()
        data=data[np.newaxis, ...]
        label = self.label[item]
        label = torch.tensor(np.array(label))
        data=self.transform(data)
        return data, label

    def __len__(self):
        return len(self.index)
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):

        return torch.from_numpy(sample)

定义数据路径 读取并预处理

In [0]:
#定义数据路径
DATASET="./dataset/"
train_INFO = pd.read_csv(os.path.join(DATASET, "train_val_new.csv"))
train_DATA= os.path.join(DATASET, 'train_val')

#读取数据并转成tensor
train_dataset = ClfDataset(train_INFO,train_DATA,crop_size=32, transform=ToTensor())

#封装成dataloader的格式
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0,drop_last=True)


定义模型

In [0]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):    # bn_size need to be 4
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv3d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv3d(bn_size * growth_rate, growth_rate,kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv3d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))

class DenseNet(nn.Module):
    def __init__(self, growth_rate=16, block_config=(4, 4, 4),
                 num_init_features=32, bn_size=4, drop_rate=0, num_classes=2):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv3d(1, num_init_features, kernel_size=3, stride=1, padding=0, bias=False)),
            ('norm0', nn.BatchNorm3d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm3d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool3d(out, kernel_size=7, stride=1).view(features.size(0), -1)
        out = self.classifier(out)
        out = F.softmax(out, dim=1)
        return out
    
    
model=DenseNet();
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


设置GPU

In [8]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

True


DenseNet(
  (features): Sequential(
    (conv0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
    (norm0): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv3d(32, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (norm2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv3d(64, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv3d(48, 64, kernel_size=(1, 1, 1), stride=(

训练模型

In [34]:
for epoch in range(10):
    model.train()
    for i,data in enumerate(tqdm(train_loader),0):
        images,labels=data
        #print(images)
        #print(labels)
        #print(i)
        images = images.type(torch.FloatTensor)
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        prediction=model(images)
        loss=criterion(prediction,labels)
        print(loss)
        loss.backward()
        optimizer.step()
        torch.save(model, 'model.pth')
    print('Finished Training')

  0%|          | 0/25 [00:00<?, ?it/s]

tensor(0.5994, device='cuda:0', grad_fn=<NllLossBackward>)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DenseNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv3d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm3d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

tensor(0.6343, device='cuda:0', grad_fn=<NllLossBackward>)


  8%|▊         | 2/25 [00:01<00:17,  1.32it/s]

...

tensor(0.5674, device='cuda:0', grad_fn=<NllLossBackward>)


 96%|█████████▌| 24/25 [00:18<00:00,  1.34it/s]

tensor(0.5676, device='cuda:0', grad_fn=<NllLossBackward>)


100%|██████████| 25/25 [00:18<00:00,  1.34it/s]

Finished Training


计算精度

In [35]:
model.eval()
correct_train = 0
size_train = 0
for i,Data in enumerate(train_loader,0):
    data,target=Data
    data = data.type(torch.FloatTensor).cuda()
    output_train = model(data).cuda()
    predict_train = torch.max(output_train.data.cuda(),1)[1]
    size_train +=target.size(0)
    correct_train +=(predict_train==target.cuda()).sum()
print('Training Accuary: %0.2f' %(100.0*correct_train/size_train), '%')
    
# correct_test = 0
# size_test=0
# for i,data,target in (test_loader,0):
#     data = data.type(torch.FloatTensor)
#     output_test = model(data)
#     predict_test = torch.max(output_test.data,1)[1]
#     size_test +=target.size(0)
#     correct_test +=(predict_test==target).sum()
# print('Testing Accuracy: %0.2f' % (100.0*correct_test/size_test), '%')

Training Accuary: 69.40 %


预测测试集

In [0]:
#定义读取格式
class testDataset(Dataset):
    def __init__(self,test_INFO,test_DATA,cropsize=32, transform=None):
        index = []
        index += list(test_INFO.index)
        self.index = tuple(sorted(index))
        self.transform = transform
        self.cropsize = cropsize

    def __getitem__(self, item):
        name = test_INFO.loc[self.index[item], 'name']
        npz_temp = np.load(os.path.join(test_DATA, '%s.npz' % name))
        seg_temp = npz_temp['seg']+0
        voxel_temp = npz_temp['voxel']
        seg_temp = seg_temp[50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2]
        voxel_temp = voxel_temp[50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2]
        data=np.multiply(seg_temp,voxel_temp)
        data = data[np.newaxis, ...]
        #seg = seg_temp[np.newaxis, ...]
        return name, data
    
    def __len__(self):
        return len(self.index)

class ToTensor(object):
"""Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):

    return torch.from_numpy(sample)

In [0]:
#定义数据路径
DATASET="./dataset/"
test_INFO = pd.read_csv(os.path.join(DATASET, "sampleSubmission.csv"))
#test_INFO = pd.read_csv("sampleSubmission(1).csv")
test_DATA= os.path.join(DATASET, 'test')
#test_DATA= 'test(1)'
#读取数据并转成tensor
test_dataset = testDataset(test_INFO,test_DATA,cropsize=32, transform=ToTensor())
#封装成dataloader的格式
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0,drop_last=False)

In [0]:
csvFile = open("test_submit.csv", "w", newline='')
writer = csv.writer(csvFile)
writer.writerow(["Id", "Predicted"])
model = torch.load('model.pth', map_location='cuda:0')
model.eval()
device = torch.device("cuda:0")
model.to(device)
with torch.no_grad():
    for ID,data in test_loader:
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        outputs = model(data)
        outputs=torch.softmax(outputs,1)
        writer.writerow([str(ID[0]), (outputs.data[0][1]).item()])
            #print(str(ID[0]), (outputs.data[0][0]).item(), (outputs.data[0][1]).item())
csvFile.close()